## Concepts

From one current user find best correlated user

Recommend 5 best rated anime from the best correlated user

Use the best rated anime from current user and recommend 5 more 

Total 10 recommendation is provided 

## Import Library

In [355]:
import pandas as pd
import string
import re

## Variables

In [356]:
target_user_id = 2
target_vote_number = 1000
min_vote_number = 10

## Loading Dataset
▪ Loading anime.csv and rating.csv into DataFrame

In [357]:
anime_data = pd.read_csv('anime.csv')
#animeDf = animeDf[:5000]
anime_data = pd.DataFrame(anime_data)
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [358]:
rating_data = pd.read_csv('rating.csv')
rating_data = rating_data[:1000000]
rating_data = pd.DataFrame(rating_data)
rating_data

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
999995,9358,28999,-1
999996,9358,29067,-1
999997,9358,29093,-1
999998,9358,29095,-1


# Data Preprocessing

In [359]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [360]:
#convert to string rather than object
anime_data['episodes'] = anime_data['episodes'].replace('Unknown', -1)
anime_data = anime_data.astype({"name":"string","genre":"string","type":"string","episodes":"int64"})

In [361]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  string 
 2   genre     12232 non-null  string 
 3   type      12269 non-null  string 
 4   episodes  12294 non-null  int64  
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), string(3)
memory usage: 672.5 KB


In [362]:
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   user_id   1000000 non-null  int64
 1   anime_id  1000000 non-null  int64
 2   rating    1000000 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


## Check Missing Value

In [363]:
rating_data.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [364]:
anime_data.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

## Drop Missing Value

In [365]:
anime_data.dropna(axis=0, inplace= True)
anime_data.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

## Check Duplicates

In [366]:
#check duplicate anime
duplicated_anime = anime_data[anime_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_anime}')

count of duplicated anime: 0


In [367]:
#check duplicate rating
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_rating}')

count of duplicated anime: 0


In [368]:
rating_data.drop_duplicates(keep='first',inplace=True)
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated anime after removing: {duplicated_rating}')

count of duplicated anime after removing: 0


## Merge Dataset by anime_id

In [369]:
raw_merged_data = pd.merge(rating_data, anime_data, on='anime_id', sort = True)
raw_merged_data

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
999981,6361,34283,7,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1,6.74,937
999982,7114,34283,6,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1,6.74,937
999983,2951,34324,-1,Chiryokumaru,Action,ONA,1,5.40,817
999984,3009,34324,6,Chiryokumaru,Action,ONA,1,5.40,817


## Drop Unnessary Column

In [370]:
#rating_y and members are not nessesary
raw_merged_data = raw_merged_data.rename(columns={"rating_x": "user_rating"})
raw_merged_data = raw_merged_data.drop('rating_y', axis=1)
raw_merged_data = raw_merged_data.drop('members', axis=1)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
...,...,...,...,...,...,...,...
999981,6361,34283,7,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1
999982,7114,34283,6,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1
999983,2951,34324,-1,Chiryokumaru,Action,ONA,1
999984,3009,34324,6,Chiryokumaru,Action,ONA,1


## Drop Unnessary Row
Drop the row that less than min_vote_number

In [371]:
grouped_data = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
trimmed_data = grouped_data[grouped_data['user_rating'] > min_vote_number]
raw_merged_data = raw_merged_data.loc[raw_merged_data['anime_id'].isin(trimmed_data.index.values.tolist())]
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
...,...,...,...,...,...,...,...
999974,9213,34240,-1,Shelter,"Music, Sci-Fi",Music,1
999975,9243,34240,9,Shelter,"Music, Sci-Fi",Music,1
999976,9278,34240,10,Shelter,"Music, Sci-Fi",Music,1
999977,9280,34240,7,Shelter,"Music, Sci-Fi",Music,1


## Clean Text

In [372]:
#before clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,rating,members
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [373]:
def clean_text(text):
    """
        data preprocessing 
    """
    
    # to lowercase
    text = text.lower()

    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
  
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [374]:
raw_merged_data['name']=raw_merged_data['name'].apply(clean_text)
anime_data['name'] = anime_data['name'].apply(clean_text)

C:\Users\Eng Lip\AppData\Local\Temp\ipykernel_15900\1514506093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_merged_data['name']=raw_merged_data['name'].apply(clean_text)


In [398]:
#after clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,rating,members
3219,34325,pokemon xyandz specials,"Action, Adventure, Comedy, Fantasy, Kids",Special,2,7.05,1454


# PART 1: User Based Correlation

## Find rating_anime_matrix

In [376]:
rating_anime_matrix = raw_merged_data.pivot_table(index='user_id', columns=['anime_id'], values = 'user_rating', fill_value= -1)
rating_anime_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,33372,33421,33524,33558,33569,33606,33964,34085,34103,34240
user_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,8,-1,-1,6,-1,6,6,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Find anime_rating_matrix

In [377]:
anime_rating_matrix = rating_anime_matrix.transpose()
anime_rating_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,9349,9350,9351,9352,9353,9354,9355,9356,9357,9358
anime_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,8,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Find the information of target_user_id

In [378]:
anime_rating_user = anime_rating_matrix[target_user_id]
anime_rating_user.sort_values(ascending= False)

anime_id
11771    10
1        -1
10391    -1
10418    -1
10417    -1
         ..
2182     -1
2179     -1
2178     -1
2176     -1
34240    -1
Name: 2, Length: 4883, dtype: int64

## Find Correlation between target_user_id with all the other users

In [379]:
similar_users = anime_rating_matrix.corrwith(anime_rating_user).dropna()
similar_users

user_id
1      -0.000410
2       1.000000
3       0.131058
5      -0.004154
7      -0.003898
          ...   
9353   -0.001604
9354   -0.000286
9355   -0.004284
9356   -0.000458
9357   -0.000821
Length: 8856, dtype: float64

## Format into Data Frame

In [380]:
sorted_similar_users = pd.DataFrame(similar_users, columns=['correlation']).sort_values(ascending= False, by= 'correlation')
sorted_similar_users

,correlation
user_id,
671,1.000000
2,1.000000
2773,0.780819
3282,0.707034
4185,0.611952
...,...
6969,-0.007188
8006,-0.007242
6569,-0.007806


## Find the best correlated user

In [381]:
#most similar user is the user that at index number 1
most_similar_user = sorted_similar_users.index.values
most_similar_user = most_similar_user[1]
most_similar_user

2

## Find 5 best rated anime from the user that have highest correlation value

In [382]:
user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)
user_recommendation[:5]

,user_id,anime_id,user_rating,name,genre,type,episodes
712107,2,11771,10,kuroko no basket,"Comedy, School, Shounen, Sports",TV,25
722175,2,12189,-1,hyouka,"Mystery, School, Slice of Life",TV,22
796804,2,16417,-1,tamako market,"Comedy, Slice of Life",TV,12


In [383]:
user_recommendation_list = user_recommendation['anime_id'].tolist()
user_recommendation_list

[11771, 12189, 16417]

# PART 2: Item Based Correlation

## Use the user's highest rated anime to recommend other related anime

In [384]:
target_user_rating = rating_data.loc[rating_data['user_id'] == target_user_id].sort_values(by = 'rating', ascending= False)
pd.DataFrame(target_user_rating.head(10))

,user_id,anime_id,rating
153,2,11771,10
154,2,12189,-1
155,2,16417,-1


In [385]:
top_rated_anime_id = target_user_rating['anime_id'].tolist()[0]
top_rated_anime_id

11771

In [386]:
ratings = rating_anime_matrix[top_rated_anime_id]
pd.DataFrame(ratings).rename(columns={top_rated_anime_id: f"{top_rated_anime_id}'s rating"})

,11771's rating
user_id,
1,-1
2,10
3,10
4,-1
5,-1
...,...
9354,-1
9355,-1
9356,-1


In [387]:
similar_animes = rating_anime_matrix.corrwith(ratings)
pd.DataFrame(similar_animes, columns=['correlation'])

,correlation
anime_id,
1,0.034432
5,-0.000947
6,0.031860
7,-0.013745
8,-0.003509
...,...
33606,0.037254
33964,0.039494
34085,-0.000088


## Identify the most correlated anime

In [388]:
sorted_similar_anime = pd.DataFrame(similar_animes, columns=['correlation']).sort_values(by= 'correlation', ascending= False)
sorted_similar_anime

,correlation
anime_id,
11771,1.000000
16894,0.832420
24415,0.595492
20583,0.458874
16916,0.377014
...,...
538,-0.023903
615,-0.024971
379,-0.025265


## Eliminate the source anime

In [389]:
sorted_similar_anime = sorted_similar_anime[1:]
sorted_similar_anime

,correlation
anime_id,
16894,0.832420
24415,0.595492
20583,0.458874
16916,0.377014
28891,0.351003
...,...
538,-0.023903
615,-0.024971
379,-0.025265


## Ensure the identified anime is popular
Higher number of votes means more popular

In [390]:
#Get number of rating for each anime
rating_votes = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
rating_votes.rename(columns={'user_rating': 'rating_count'})

,rating_count
anime_id,
1,2115
5,994
6,1487
7,361
8,61
...,...
33606,11
33964,30
34085,11


In [391]:
similar_animes_ratings = sorted_similar_anime.join(rating_votes['user_rating']).sort_values(by = 'correlation', ascending = False)
similar_animes_ratings

,correlation,user_rating
anime_id,,
16894,0.832420,1063
24415,0.595492,598
20583,0.458874,865
16916,0.377014,231
28891,0.351003,459
...,...,...
538,-0.023903,182
615,-0.024971,67
379,-0.025265,397


## Get the anime that have higher votes and have higher correlation

In [392]:
similar_popular_animes = similar_animes_ratings.loc[similar_animes_ratings['user_rating']>=target_vote_number].dropna()
similar_popular_animes

,correlation,user_rating
anime_id,,
16894,0.832420,1063
18507,0.333406,1195
20507,0.328257,1984
14513,0.318035,1258
9919,0.317774,2553
...,...,...
376,0.001724,1090
32,0.001169,1401
59,-0.003294,1636


In [393]:
most_similar_popular_animes = similar_popular_animes[:5]
most_similar_popular_animes_list = most_similar_popular_animes.index.to_list()
most_similar_popular_animes_list

[16894, 18507, 20507, 14513, 9919]

# Result Showing

In [394]:
final_recommendation_result = user_recommendation_list.copy()
final_recommendation_result += most_similar_popular_animes_list.copy()
final_recommendation_result

[11771, 12189, 16417, 16894, 18507, 20507, 14513, 9919]

## Show Target User's Rating

In [395]:
target_user = rating_data.loc[rating_data['user_id'] == target_user_id].sort_values(by= 'rating', ascending= False)
pd.DataFrame(target_user.head(10))

,user_id,anime_id,rating
153,2,11771,10
154,2,12189,-1
155,2,16417,-1


## Show Target Anime's information 

In [396]:
target_anime = anime_data.loc[anime_data['anime_id'] == top_rated_anime_id]
pd.DataFrame(target_anime)

,anime_id,name,genre,type,episodes,rating,members
122,11771,kuroko no basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315


## Show Final Recommendation Result

In [397]:
recommended_anime = anime_data.loc[anime_data['anime_id'].isin(final_recommendation_result)]
pd.DataFrame(recommended_anime)

,anime_id,name,genre,type,episodes,rating,members
72,16894,kuroko no basket 2nd season,"Comedy, School, Shounen, Sports",TV,25,8.58,243325
122,11771,kuroko no basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315
268,14513,magi the labyrinth of magic,"Action, Adventure, Fantasy, Magic, Shounen",TV,25,8.24,317513
335,12189,hyouka,"Mystery, School, Slice of Life",TV,22,8.17,372246
339,20507,noragami,"Action, Adventure, Shounen, Supernatural",TV,12,8.17,515378
643,9919,ao no exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823
1115,18507,free,"Comedy, School, Slice of Life, Sports",TV,12,7.67,265791
2133,16417,tamako market,"Comedy, Slice of Life",TV,12,7.35,128529
